In [18]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt


np.random.seed(88)

### Year 0 Expenses
- **Seismic and Lease Costs**: These depend on the number of wells. Leased acres per well follow a normal distribution (mean = 600, SD = 50), with a price of $960 per acre. Seismic sections per well also follow a normal distribution (mean = 3, SD = 0.35), costing $43,000 per section.
- **Completion Costs**: If the well is not dry, preparation for oil production incurs a normally distributed cost (mean = $390,000, SD = $50,000).
- **Professional Overhead**: Project team salary and benefits per well follow a triangular distribution (min = $172,000, most likely = $215,000, max = $279,500). These costs apply during Year 0 but cease if the well is dry.

The following code represents the year 0 expense for a **wet** well:

In [ ]:
total_cost_wet = []
for i in range(10000):
    # Seismic and Lease Costs
    leased = np.random.normal(loc = 600, scale = 50)
    seismic = np.random.normal(loc = 3, scale = .35)

    # Completion Costs
    completion = np.random.normal(loc = 390000, scale = 50000)

    # Professional Overhead
    overhead = np.random.triangular(left = 172000, mode = 215000, right = 279500)

    # Total Costs
    total_cost_wet.append( (leased*960) + (seismic*43000) + completion + overhead )


The following code represents the year 0 expense for a **dry** well:

In [ ]:
total_cost_dry = []
for i in range(10000):
    # Seismic and Lease Costs
    leased = np.random.normal(loc = 3, scale = .35)
    seismic = np.random.normal(loc = 600, scale = 50)

    # Professional Overhead
    overhead = np.random.triangular(left = 172000, mode = 215000, right = 279500)

    # Total Costs
    total_cost_dry.append( (leased*960) + (seismic*43000) + overhead )

### Production Risk
- **Initial Production**: The starting production rate in Year 1, following a Lognormal distribution (mean = 420 BOPD, SD = 120 BOPD), with an underlying Normal distribution (mean = 6, SD = 0.28).
- **Decline Rate**: The annual percentage decrease in production, uniformly distributed between 15% and 32%, remaining constant for each well's lifespan.
- A correlation coefficient of 0.64 is imposed between initial production & decline rate in the simulation.

Production rates in BOPD for our model are calculated by:   
$$
\text{New Rate} = (1 - \text{Decline Rate}) \times \text{Previous Rate}
$$
Yearly production volumes in barrels of oil are approximated as:   
$$
\text{Oil Volume} = \frac{365 \times (\text{Rate}_{\text{start}} + \text{Rate}_{\text{end}})}{2}
$$

In [33]:
# Initial Production
init_production = np.random.lognormal(mean = 6, sigma = .28, size=10000)  #I'm pretty sure this is wrong but idk how to incorportate the underlying normal dist.

# Decline Rate
decline_rate = np.random.uniform(low = .15, high = .32, size=10000)

# New Rate
new_rate = (1 - decline_rate) * init_production

R = np.array([[1, .64], [.64, 1]])
U = sp.linalg.cholesky(R, lower = False)
both = np.column_stack((init_production, decline_rate))
X1X2 = U @ both.T
X1X2 = np.transpose(X1X2)


# Oil Volume
# oil_volume = 365 * ()


### Revenue Risk
- **Net Revenue Interest**: The company retains a portion of revenue after royalties, modeled as a Normal distribution (mean 75%, SD 2%) per well.

### Operating Expenses
- **Operating Costs**: Expenses for manpower and hardware are modeled as a Normal distribution with a mean of $2.25 per barrel (SD $0.30). Costs are the same for all wells in a given year but vary annually.
- **Severance Taxes**: A fixed 4.6% tax on revenue, applied after the NRI.

### Net Present Value Calculation
NPV is computed by summing all revenues and expenses per year, discounted at a 10% WACC, and subtracting initial costs. The formula used is:
$$
\text{NPV} = -\text{Initial Costs} + \frac{\text{FNR}_{\text{year 1}} }{1 + \text{WACC}} + \frac{\text{FNR}_{\text{year 2}} }{(1 + \text{WACC})^2} + ... + \frac{\text{FNR}_{\text{year 15}} }{(1 + \text{WACC})^{15}}
$$
Where FNR is the final net revenues during the specified year.